# Optimization with DSPY

In [10]:
import dspy
from dspy.evaluate import Evaluate
from typing import Literal
import pandas as pd
import numpy as np
import time
import bert_score
import warnings
from transformers import logging
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

### Settings

In [ ]:
wait_time = 0 # impostare se ci sono rate limits, altrimenti lasciare 0
debug_mode = False

In [3]:
data_folder = './data/'
result_folder = './results/'
prompt_folder = './prompts/'
optimizers_folder = './optimizers/'

### LLM

In [4]:
model_name = 'mistral'
lm = dspy.LM('mistral/mistral-large-latest', api_key = os.getenv("MISTRAL_API_KEY")) # Default temperature = 0.0
dspy.configure(lm = lm)

In [5]:
# TEST
# model_name = 'chatgpt'
# dspy.LM('openai/gpt-4o-mini', api_key = os.getenv("OPENAI_API_KEY"))
# dspy.configure(lm = lm)

In [6]:
# TEST
#model_name = 'llama'
#lm = dspy.LM('ollama_chat/llama3.2', api_base='http://localhost:11434', api_key='')
#dspy.configure(lm = lm)

In [7]:
# TEST
# model_name = 'qwen'
# lm = dspy.LM(
#     'openai/qwen2.5-72b-instruct-large-latest', 
#     api_key = os.getenv('DASHSCOPE_API_KEY'),
#     api_base = 'https://dashscope-intl.aliyuncs.com/compatible-mode/v1'
# )
# dspy.configure(lm = lm)

In [8]:
# TEST
# model_name = 'deepseek'
# lm = dspy.LM(
#     'deepseek/deepseek-chat', 
#     api_key = os.getenv('DEEPSEEK_API_KEY'),
#     api_base = 'https://api.deepseek.com'
# )
# dspy.configure(lm = lm)

In [9]:
# TEST
print(lm("What is 2+2?"))

['The sum of 2 + 2 is 4. Here it is:\n\n2\n+2\n____\n4']


### Prompt ToT

In [11]:
with open(prompt_folder + 'prompt_tot.txt') as f:
    tree_of_thoughts = f.read()
print(tree_of_thoughts)

Follow this procedure:

Imagine three different experts are answering this question.
They will brainstorm the answer step by step reasoning carefully and taking all facts into consideration.
All experts will write down 1 step of their thinking, then share it with the group.
They will each critique their response, and the all the responses of others.
They will check their answer on based on the nature of the language and intent.
Then all experts will go on to the next step and write down this step of their thinking.
They will keep going through steps until they reach their conclusion taking into account the thoughts of the other experts.
If at any time they realise that there uncertainty in their logic they will backtrack to where that uncertainty occurred. 
If any expert realises they're wrong at any point then they acknowledges this and start another train of thought.
Each expert will assign a likelihood of their current assertion being correct.
Continue until the experts agree on the

# Stage 1: High Level Categorization

In [12]:
def optimized_classify_stage_1(optimizer, df_hate, result_file, initialize = False):
    
    if initialize:
        # Regenerate output file
        df = pd.DataFrame(columns = ['post_id', 'class','confidence','explanation'])
        df.to_csv(result_file, encoding = "utf-8", index = False)
    else:
        df_elab = pd.read_csv(result_file, encoding = "utf-8")
        df_hate = df_hate[~df_hate['post_id'].isin(df_elab['post_id'])]

    for idx, row in df_hate.iterrows():
    
        try:            
            time.sleep(wait_time)
            classification = optimizer(post = row['post'])
        
            # Classification stage 1
            new_row = {
                'post_id': row['post_id'], 
                'class': classification.hate_class, 
                'confidence': classification.confidence,
                'explanation': classification.explanation
            }
            df = pd.DataFrame([new_row])
            df.to_csv(result_file, encoding = "utf-8", mode='a', index=False, header=False)
                      
        except:
          print("An exception occurred " +  str(row['post_id']) ) 

### Signature

In [13]:
with open(prompt_folder + 'prompt_stg1.txt') as f:
    prompt_stg1 = f.read()

In [14]:
prompt_stg1 = prompt_stg1.replace("{tree_of_thoughts}", tree_of_thoughts)

In [15]:
print(prompt_stg1)

Your task is to decide whether the following text can be classified an implicit hate speech or not implicit hate speech.

Consider the following definition of hate speech:

Hate speech is content that targets individuals or groups with abuse based on their perceived membership in protected categories, including but not limited to race, ethnicity, national origin, caste, sexual orientation, gender, gender identity, religious affiliation, age, disability, or serious disease. 
Specifically, hate speech can contain:

- Hateful References: Content referencing forms of violence or violent events where a protected category was the primary target, intended to harass (e.g., genocides like the Holocaust, lynchings).
- Incitement: Content inciting fear, spreading fearful stereotypes, or encouraging others to harass or discriminate against members of protected categories (e.g., asserting that members of a religious group are terrorists, urging others to harass individuals wearing religious symbols

In [16]:
class ClassifyHate(dspy.Signature):
    prompt_stg1

    post: str = dspy.InputField()
    hate_class: Literal["implicit_hate", "not_hate"]  = dspy.OutputField()
    confidence: float = dspy.OutputField()
    explanation : str = dspy.OutputField()

classify_hate = dspy.Predict(ClassifyHate)

### Metric

In [17]:
def classify_hate_metric(example, prediction, trace=None):
    time.sleep(wait_time)
    return prediction.hate_class == example.hate_class

### Trainset

In [18]:
df_train = pd.read_csv(data_folder + "implicit_hate_train_stg1.csv", encoding="utf-8")

In [19]:
trainset = []
for index, row in df_train.sample(100).iterrows():
    example = dspy.Example(post = row['post'], hate_class = row['class']).with_inputs("post")
    trainset.append(example)

### Evaluator

In [43]:
evaluator = Evaluate(devset = trainset, num_threads = 1, display_progress = True, display_table = 5)
evaluator(classify_hate, metric = classify_hate_metric)

Average Metric: 56.00 / 100 (56.0%): 100%|███████████████████████████████████████████| 100/100 [01:40<00:00,  1.01s/it]

2025/01/08 12:03:10 INFO dspy.evaluate.evaluate: Average Metric: 56 / 100 (56.0%)


,post,example_hate_class,pred_hate_class,confidence,explanation,classify_hate_metric
0,lol ! you guys talk so good . words use to make the fancy statemen...,not_hate,implicit_hate,0.75,"The post contains sarcastic language with phrases like ""lol"" and ""...",
1,a petition stop christian court martial policy-check out this arti...,not_hate,not_hate,0.95,The post is sharing an article and expressing a political viewpoin...,✔️ [True]
2,uk telegraph : racial conflicts will dominate the 21st century via,not_hate,implicit_hate,0.75,The post suggests that racial conflicts will be a significant issu...,
3,what have black people contributed to science ?,implicit_hate,implicit_hate,0.85,The post implies a stereotype or prejudice against black people by...,✔️ [True]
4,white supremacists are glad boehners leaving via,not_hate,implicit_hate,0.85,"The post mentions ""white supremacists"" in a context that suggests ...",


56.0

In [16]:
optimized_classify_hate.save(optimizers_folder + model_name + '_' + 'optimizer_stg1_v1.json')

### Optimization with MIPRO (prompt and examples)

In [45]:
classify_hate_optimizer = dspy.MIPROv2(
    metric = classify_hate_metric, 
    auto = "heavy"
)

optimized_classify_hate = classify_hate_optimizer.compile(
    classify_hate, 
    trainset = trainset, 
    max_labeled_demos = 8, 
    max_bootstrapped_demos = 8
)

2025/01/08 12:06:33 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING HEAVY AUTO RUN SETTINGS:
num_trials: 50
minibatch: True
num_candidates: 38
valset size: 80



Projected Language Model (LM) Calls

Based on the parameters you have set, the maximum number of LM calls is projected as follows:

- Prompt Generation: 10 data summarizer calls + 38 * 1 lm calls in program + (2) lm calls in program-aware proposer = 50 prompt model calls
- Program Evaluation: 25 examples in minibatch * 50 batches + 80 examples in val set * 6 full evals = 1730 LM Program calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of program calls * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expecta

2025/01/08 12:06:37 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/01/08 12:06:37 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/01/08 12:06:37 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=38 sets of demonstrations...


Bootstrapping set 1/38
Bootstrapping set 2/38
Bootstrapping set 3/38


 45%|█████████████████████████████████████▎                                             | 9/20 [00:27<00:34,  3.10s/it]


...

2025/01/08 12:45:52 ERROR dspy.utils.parallelizer: Error processing item Example({'post': 'didnt blue lives matter just a second ago ? oh thats right  cops kill white people for no reason too sometimes . youd think you just found this out ? or are you all just bigots ?', 'hate_class': 'implicit_hate'}) (input_keys={'post'}): litellm.RateLimitError: RateLimitError: MistralException - Error code: 429 - {'message': 'Requests rate limit exceeded'}. Set `provide_traceback=True` to see the stack trace.


Average Metric: 15.00 / 24 (62.5%): 100%|██████████████████████████████████████████████| 25/25 [00:21<00:00,  1.14it/s]

2025/01/08 12:45:59 INFO dspy.evaluate.evaluate: Average Metric: 15.0 / 25 (60.0%)
2025/01/08 12:45:59 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 60.0 on minibatch of size 25 with parameters ['Predictor 0: Instruction 12', 'Predictor 0: Few-Shot Set 37'].
2025/01/08 12:45:59 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [76.0, 80.0, 68.0, 56.0, 72.0, 60.0, 80.0, 80.0, 76.0, 64.0, 68.0, 64.0, 48.0, 68.0, 72.0, 60.0, 80.0, 72.0, 84.0, 68.0, 68.0, 84.0, 68.0, 76.0, 68.0, 60.0, 56.0, 84.0, 68.0, 68.0, 68.0, 52.0, 68.0, 72.0, 84.0, 84.0, 76.0, 68.0, 68.0, 64.0, 64.0, 64.0, 80.0, 56.0, 60.0, 72.0, 68.0, 84.0, 64.0, 60.0]
2025/01/08 12:45:59 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [55.0, 70.0, 63.75, 73.75, 71.25]
2025/01/08 12:45:59 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 73.75
2025/01/08 12:45:59 INFO dspy.teleprompt.mipro_optimizer_v2: =============================


2025/01/08 12:45:59 INFO dspy.teleprompt.mipro_o


Average Metric: 57.00 / 80 (71.2%): 100%|██████████████████████████████████████████████| 80/80 [00:51<00:00,  1.57it/s]

2025/01/08 12:46:50 INFO dspy.evaluate.evaluate: Average Metric: 57 / 80 (71.2%)
2025/01/08 12:46:50 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [55.0, 70.0, 63.75, 73.75, 71.25, 71.25]
2025/01/08 12:46:50 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 73.75
2025/01/08 12:46:50 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/01/08 12:46:50 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/01/08 12:46:50 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 73.75!


In [18]:
optimized_classify_hate.save(optimizers_folder + model_name + '_' + 'optimizer_stg1.json')

### Run optimized classifier

In [21]:
optimizer_file_name = optimizers_folder + model_name + '_' + 'optimizer_stg1.json'
result_file = result_folder + model_name + "_" + "result_optimizer_stg1.csv"
df_hate = pd.read_csv(data_folder + "implicit_hate_test_stg1.csv", encoding="utf-8")

In [22]:
if debug_mode:
    df_hate = df_hate.sample(2)

In [23]:
optimized_classify_hate = classify_hate
optimized_classify_hate.load(optimizer_file_name)
optimized_classify_stage_1(optimized_classify_hate, df_hate, result_file, True)

# Stage 2: Fine-Grained Implicit Hate

In [24]:
def optimized_classify_stage_2(optimizer, df_hate, result_file, initialize = False):
    
    if initialize:
        # Regenerate output file
        df = pd.DataFrame(columns = ['post_id', 'implicit_class','confidence','explanation'])
        df.to_csv(result_file, encoding = "utf-8", index = False)
    else:
        df_elab = pd.read_csv(result_file, encoding = "utf-8")
        df_hate = df_hate[~df_hate['post_id'].isin(df_elab['post_id'])]

    for idx, row in df_hate.iterrows():
    
        try:            
            time.sleep(wait_time)
            classification = optimizer(post = row['post'])
        
            # Classification stage 2
            new_row = {
                'post_id': row['post_id'], 
                'implicit_class': classification.implicit_class, 
                'confidence': classification.confidence,
                'explanation': classification.explanation
            }
            df = pd.DataFrame([new_row])
            df.to_csv(result_file, encoding = "utf-8", mode='a', index=False, header=False)
                      
        except:
          print("An exception occurred " +  str(row['post_id']) ) 

### Signature

In [25]:
with open(prompt_folder + 'prompt_stg2.txt') as f:
    prompt_stg2 = f.read()

In [26]:
prompt_tot_stg2 = prompt_stg2.replace("{tree_of_thoughts}", tree_of_thoughts)

In [27]:
print(prompt_tot_stg2)

The following text has been categorized as "implicit hate speech", a more complex and subtle attack on a person or a group of people based on their protected characteristics without relying on explicit keywords.

Evaluate the text against the following taxonomy to determine the most likely class.
Class taxonomy of implicit hate speech:

1. White Grievance: includes frustration over a minority group's perceived privilege and casting majority groups as the real victims of racism. This language is linked to extremist behavior and support for violence. An example is 'Black lives matter and white lives don't? Sounds racist.'
2. Incitement to Violence: includes flaunting ingroup unity and power or elevating known hate groups and ideologies. Phrases like 'white brotherhood' operate in the former manner, while statements like 'Hitler was Germany â€“ Germans shall rise again!' operate in the latter, elevating nationalism and Nazism. Article 20 of the UN International Covenant on Civil and Polit

In [28]:
class ClassifyImplicitHate(dspy.Signature):
    prompt_tot_stg2

    post: str = dspy.InputField()
    implicit_class: Literal['white_grievance', 'irony', 'stereotypical', 'incitement', 'other', 'threatening', 'inferiority', 'other'] = dspy.OutputField()
    confidence: float = dspy.OutputField()
    explanation : str = dspy.OutputField()

classify_implicit_hate = dspy.Predict(ClassifyImplicitHate)

### Metric

In [29]:
def classify_implicit_hate_metric(example, prediction, trace=None):
    time.sleep(wait_time)
    return prediction.implicit_class == example.implicit_class

### Trainset

In [30]:
df_train_stg2 = pd.read_csv(data_folder + "implicit_hate_train_stg2.csv", encoding="utf-8")

In [31]:
len(df_train_stg2)

1000

In [32]:
df_train_stg2['implicit_class'].unique()

array(['inferiority', 'incitement', 'irony', 'white_grievance',
       'threatening', 'stereotypical', 'other'], dtype=object)

In [33]:
# Select n samples for each class
n = 50
df_train = pd.DataFrame()
for implicit_class in df_train_stg2['implicit_class'].unique():
    df_class = df_train_stg2[df_train_stg2['implicit_class'] == implicit_class]
    df_class = df_class.sample(np.min((n,len(df_class))))
    df_train = pd.concat([df_train, df_class])

In [34]:
trainset = []
for index, row in df_train.iterrows():
    example = dspy.Example(post = row['post'], implicit_class = row['implicit_class']).with_inputs("post")
    trainset.append(example)

### Evaluator

In [41]:
evaluator = Evaluate(devset = trainset, num_threads = 1, display_progress = True, display_table = 5)
evaluator(classify_implicit_hate, metric = classify_implicit_hate_metric)

Average Metric: 105.00 / 313 (33.5%): 100%|██████████████████████████████████████████| 313/313 [05:14<00:00,  1.00s/it]

2025/01/08 17:33:25 INFO dspy.evaluate.evaluate: Average Metric: 105 / 313 (33.5%)


,post,example_implicit_class,pred_implicit_class,confidence,explanation,classify_implicit_hate_metric
0,manages to maintain it's culture. guess it's well out the way of t...,inferiority,stereotypical,0.75,The post suggests that a certain culture is maintained because it ...,
1,libertarian solution : more third world immigration,inferiority,irony,0.85,The post appears to be using irony to highlight a contrast between...,
2,why do colored people eagerly run to white countries to be oppress...,inferiority,white_grievance,0.95,"The post suggests a grievance from a white perspective, implying t...",
3,white power black parasite. blacks have contributed nothing to hum...,inferiority,incitement,0.95,The post contains explicit racist language and derogatory remarks ...,
4,i have seen this woman on youtube she is fantastic and so knowledg...,inferiority,stereotypical,0.85,The post stereotypes Muslims by referring to their presence in the...,


33.55

### Optimization with MIPRO (prompt and examples)

In [15]:
print('Samples in trainset: {}'.format(len(trainset)))

classify_implicit_hate_optimizer = dspy.MIPROv2(
    metric = classify_implicit_hate_metric, 
    auto = "heavy",
    num_threads = 2
)

optimized_classify_implicit_hate = classify_implicit_hate_optimizer.compile(
    classify_implicit_hate, 
    trainset = trainset, 
    max_labeled_demos = 20, 
    max_bootstrapped_demos = 20
)

2025/01/10 16:14:50 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING HEAVY AUTO RUN SETTINGS:
num_trials: 50
minibatch: True
num_candidates: 38
valset size: 250



Projected Language Model (LM) Calls

Based on the parameters you have set, the maximum number of LM calls is projected as follows:

- Prompt Generation: 10 data summarizer calls + 38 * 1 lm calls in program + (2) lm calls in program-aware proposer = 50 prompt model calls
- Program Evaluation: 25 examples in minibatch * 50 batches + 250 examples in val set * 6 full evals = 2750 LM Program calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of program calls * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expect

2025/01/10 16:14:54 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/01/10 16:14:54 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/01/10 16:14:54 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=38 sets of demonstrations...


Bootstrapping set 1/38
Bootstrapping set 2/38
Bootstrapping set 3/38


 52%|██████████████████████████████████████████▉                                       | 33/63 [04:23<03:59,  7.97s/it]


...

2025/01/10 18:40:46 INFO dspy.evaluate.evaluate: Average Metric: 18 / 25 (72.0%)
2025/01/10 18:40:46 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 72.0 on minibatch of size 25 with parameters ['Predictor 0: Instruction 29', 'Predictor 0: Few-Shot Set 19'].
2025/01/10 18:40:46 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [44.0, 60.0, 32.0, 36.0, 56.0, 48.0, 48.0, 48.0, 28.0, 48.0, 60.0, 72.0, 64.0, 36.0, 48.0, 56.0, 24.0, 40.0, 60.0, 56.0, 48.0, 44.0, 64.0, 44.0, 44.0, 40.0, 44.0, 36.0, 52.0, 48.0, 48.0, 32.0, 44.0, 32.0, 56.0, 52.0, 44.0, 32.0, 40.0, 52.0, 28.0, 32.0, 36.0, 52.0, 44.0, 64.0, 56.0, 64.0, 56.0, 72.0]
2025/01/10 18:40:46 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [50.8, 49.2, 52.8, 46.8, 48.4]
2025/01/10 18:40:46 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 52.8
2025/01/10 18:40:46 INFO dspy.teleprompt.mipro_optimizer_v2: =============================


2025/01/10 18:40:46 INFO dspy.teleprompt.mipro_optimiz


Average Metric: 90.00 / 187 (48.1%):  75%|████████████████████████████████▏          | 187/250 [02:33<01:22,  1.32s/it]

2025/01/10 18:43:21 ERROR dspy.utils.parallelizer: Error processing item Example({'post': "4396 white murderers  5375 black murderers . all the while blacks are 13 % of the country's population .", 'implicit_class': 'stereotypical'}) (input_keys={'post'}): litellm.RateLimitError: RateLimitError: MistralException - Error code: 429 - {'message': 'Requests rate limit exceeded'}. Set `provide_traceback=True` to see the stack trace.


Average Metric: 125.00 / 249 (50.2%): 100%|██████████████████████████████████████████| 250/250 [03:22<00:00,  1.24it/s]

2025/01/10 18:44:09 INFO dspy.evaluate.evaluate: Average Metric: 125.0 / 250 (50.0%)
2025/01/10 18:44:09 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [50.8, 49.2, 52.8, 46.8, 48.4, 50.0]
2025/01/10 18:44:09 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 52.8
2025/01/10 18:44:09 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/01/10 18:44:09 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/01/10 18:44:09 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 52.8!


In [20]:
optimized_classify_implicit_hate.save(optimizers_folder + model_name + '_' + 'optimizer_stg2_v2.json')

### Run optimized classifier

In [36]:
optimizer_file_name = optimizers_folder + model_name + '_' + 'optimizer_stg2.json'
result_file = result_folder + model_name + "_" + "result_optimizer_stg2.csv"
df_hate_stg2 = pd.read_csv(data_folder + "implicit_hate_test_stg2.csv", encoding="utf-8")

In [37]:
if debug_mode:
    df_hate_stg2 = df_hate_stg2.sample(2)

In [38]:
# Reload saved optimizer
optimized_classify_implicit_hate = classify_implicit_hate
optimized_classify_implicit_hate.load(optimizer_file_name)
optimized_classify_stage_2(optimized_classify_implicit_hate, df_hate_stg2, result_file, True)

## Optimization stage 3

In [39]:
warnings.filterwarnings("ignore")
logging.set_verbosity_error()

In [40]:
def optimized_classify_stage_3(optimizer, df_hate, result_file, initialize = False):
    
    if initialize:
        # Regenerate output file
        df = pd.DataFrame(columns = ['post_id', 'target', 'implied_statement'])
        df.to_csv(result_file, encoding = "utf-8", index = False)
    else:
        df_elab = pd.read_csv(result_file, encoding = "utf-8")
        df_hate = df_hate[~df_hate['post_id'].isin(df_elab['post_id'])]

    for idx, row in df_hate.iterrows():
    
        try:            
            time.sleep(wait_time)
            classification = optimizer(post = row['post'])
        
            # Classification stage 3
            new_row = {
                'post_id': row['post_id'], 
                'target': classification.targeted_group, 
                'implied_statement': classification.implied_meaning
            }
            df = pd.DataFrame([new_row])
            df.to_csv(result_file, encoding = "utf-8", mode='a', index=False, header=False)
                      
        except:
          print("An exception occurred " +  str(row['post_id']) ) 

### Signature

In [41]:
with open(prompt_folder + 'prompt_stg3.txt') as f:
    prompt_stg3 = f.read()

In [42]:
prompt_tot_stg3 = prompt_stg3.replace("{tree_of_thoughts}", tree_of_thoughts)

In [43]:
print(prompt_tot_stg3)

The following text has been categorized as "implicit hate speech", a more complex and subtle attack on a person or a group of people based on their protected characteristics without relying on explicit keywords.

Your job is to infer both the targeted group (GROUP) and the stereotype, characteristic, or implication about that group expressed in the text. 
You have two tasks:

Task 1: Identify the targeted group (GROUP).
The group might be defined by characteristics such as ethnicity, religion, class, sexual orientation, immigration status, or similar traits. Examples of groups include:

-Black folks
-Asian folks
-Muslims
-Jews
-Latino/Latina folks
-Immigrants


Task 2: Determine what stereotype, characteristic, or action is implied about the identified group.
Using the group identified in Task 1, describe the implication using a simple phrase. Avoid copying text directly from the text. Examples of such phrases include:

-[GROUP] do/does [action]
-[GROUP] are [characteristic]
-[GROUP] k

In [44]:
class IdentifyImplicitMeaning(dspy.Signature):
    prompt_tot_stg3

    post: str = dspy.InputField()
    targeted_group: str = dspy.OutputField()
    implied_meaning : str = dspy.OutputField()

indentify_implicit_meaning = dspy.Predict(IdentifyImplicitMeaning)

### Metric

In [45]:
def implied_meaning_metric(example, prediction, trace=None):
    time.sleep(1)
    P, R, F1 = bert_score.score([example.implied_meaning], [prediction.implied_meaning], lang='en', verbose=False)
    bert_f1 = F1.mean().item()

    return bert_f1 > 0.90

### Training data

In [46]:
df_train = pd.read_csv(data_folder + "implicit_hate_train_stg3.csv", encoding="utf-8")

In [47]:
# Load the trainset of n samples
n = 100
trainset = []
for index, row in df_train.sample(n).iterrows():
    example = dspy.Example(post = row['post'], targeted_group = row['target'], implied_meaning = row['implied_statement']).with_inputs("post")
    trainset.append(example)

### Evaluator

In [ ]:
evaluator = Evaluate(devset = trainset, num_threads = 1, display_progress = True, display_table = 5)
evaluator(indentify_implicit_meaning, metric = implied_meaning_metric)

### Optimization with MIPRO (prompt and examples)

In [ ]:
print('Samples in trainset: {}'.format(len(trainset)))

identify_implicit_meaning_optimizer = dspy.MIPROv2(
    metric = implied_meaning_metric, 
    auto = "heavy"
)

optimized_identify_implicit_meaning = identify_implicit_meaning_optimizer.compile(
    indentify_implicit_meaning, 
    trainset = trainset, 
    max_labeled_demos = 20, 
    max_bootstrapped_demos = 20    
)

In [ ]:
optimized_identify_implicit_meaning.save(optimizers_folder + model_name + '_' + 'optimizer_stg3.json')

### Run optimized classifier

In [49]:
optimizer_file_name = optimizers_folder + model_name + '_' + 'optimizer_stg3.json'
result_file = result_folder + model_name + '_' + 'result_optimizer_stg3.csv'
df_hate = pd.read_csv(data_folder + "implicit_hate_test_stg3.csv", encoding = "utf-8")
df_hate = df_hate[['post_id', 'post']].drop_duplicates()

In [50]:
if debug_mode:
    df_hate = df_hate.sample(2)

In [52]:
optimized_identify_implicit_meaning = indentify_implicit_meaning
optimized_identify_implicit_meaning.load(optimizer_file_name)
optimized_classify_stage_3(optimized_identify_implicit_meaning, df_hate, result_file, True)